<a href="https://colab.research.google.com/github/tired2onfire/SA/blob/master/SA_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/tired2onfire/SA.git

Cloning into 'SA'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 22 (delta 2), reused 5 (delta 0), pack-reused 0
Unpacking objects: 100% (22/22), done.


In [ ]:
!pip install transformers

In [ ]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import json

In [ ]:
with open('SA/friends_train.json', 'r', encoding='UTF-8') as f:
    json_train = json.load(f)

train = pd.array(json_train)
print('training set: ', train.shape)

with open('SA/friends_dev.json', 'r', encoding='UTF-8') as f:
    json_dev = json.load(f)

dev = pd.array(json_dev)
print('dev set: ', dev.shape)

training set:  (720,)
dev set:  (80,)


/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [40]:
exam_mode = True
sentence_len = 2
flag_speaker = True
MAX_LEN = 0
PRETRAINED_MODEL = 'bert-base-multilingual-cased'
BATCH_SIZE = 36
EPOCHS = 4
SEED = 42

In [ ]:
test = []
sen_set = []
i = 0

if exam_mode: 
    with open('SA/SA_EN_Friends.csv', 'r', encoding='utf-8') as f:
        data = pd.read_csv(f)

    for j in range(len(data)):
        if(data['i_dialog'][j] != i):
            i = data['i_dialog'][j]
            test.append(sen_set)
            sen_set = []

        sen = {'speaker':data['speaker'][j], 'utterance':data['utterance'][j], 'emotion': 'neutral'}  
        sen_set.append(sen)
    test.append(sen_set)
else:  
    with open('SA/friends_test.json', 'r', encoding='UTF-8') as f:
        test = json.load(f)

test = pd.array(test)
print('test set: ', test.shape)

In [42]:
def labeltoint(lbl):
    return {'neutral': 0, 'non-neutral': 1, 'joy': 2, 'sadness': 3, 'anger': 4, 'surprise': 5, 'fear': 6, 'disgust': 7}[lbl]

def inttolabel(lbl):
    return {0: 'neutral', 1: 'non-neutral',2: 'joy', 3: 'sadness', 4: 'anger', 5: 'surprise', 6: 'fear', 7: 'disgust'}[lbl]

def speakertoint(lbl):
    return {'Chandler': 0, 'Joey': 1, 'Ross': 2, 'Rachel': 3, 'Monica': 4, 'Phoebe': 5, 'Other': 6}[lbl]

def inttospeaker(lbl):
    return {0: 'Chandler', 1: 'Joey',2: 'Ross', 3: 'Rachel', 4: 'Monica', 5: 'Phoebe', 6:'Other'}[lbl]

def speakerCheck(lbl):
    if lbl == 'Chandler' or lbl == 'Joey' or lbl == 'Ross' or lbl == 'Rachel' or lbl == 'Monica' or lbl == 'Phoebe':
        return lbl
    else:
       return 'Other'



def replaceunicode(text):
    text = text.replace('\u0085', " ")
    text = text.replace('\u0091', "'")
    text = text.replace('\u0092', "'")
    text = text.replace('\u0093', "'")
    text = text.replace('\u0094', "'")
    text = text.replace('\u0097', ", ")
    text = text.replace('\u2019', "'")
    text = text.replace('\u00a0', "")
    text = text.replace('\u2026', "...")
    text = text.replace('\u2014', ". ")
    text = text.replace('\u00e9', "")
    text = text.replace('\u0096', ".")
    text = text.replace('\u00e8', "!")
    return text


def Make_Sentence(df, speaker, sentence_len):

    empty_str = ""
    result = ""

    empty_len = sentence_len - len(df)

    for s, p in zip(df, speaker):

        if flag_speaker:  
            if p in {'Chandler', 'Joey', 'Ross', 'Rachel', 'Monica', 'Phoebe'}:
              result =  result + ' {0}'.format(p)

        result = result + " " + replaceunicode(s) + " [SEP]"

    result = empty_str + result
    result = "[CLS]" + result

    return result


def makeTalkingSet(trainingSet):
    sentences = []
    labels = []
    speaker = []
    for talking_set in trainingSet:
        df_talk = pd.DataFrame(talking_set)
        
        for i in range(df_talk.index.size):
            sentence = df_talk.iloc[max(0,i-sentence_len+1):i+1]['utterance']
            sentence_speaker = df_talk.iloc[max(0,i-sentence_len+1):i+1]['speaker']
            dummy_sen = Make_Sentence(sentence, sentence_speaker, sentence_len)
            sentences.append(dummy_sen)
            labels.append(labeltoint(df_talk.iloc[i]['emotion']))
            speaker.append(speakertoint(speakerCheck(df_talk.iloc[i]['speaker'])))
    
    sentences = pd.Series(sentences)

    return sentences, labels, speaker

In [ ]:
sentences_train, labels_train, speaker_train = makeTalkingSet(train)
sentences_dev, labels_dev, speaker_dev = makeTalkingSet(dev)
sentences_test, labels_test, speaker_test = makeTalkingSet(test)

print(sentences_train.shape)
print(sentences_train[:10])
print(len(labels_train))
print(labels_train[:100]

In [ ]:
tokenizer = BertTokenizer.from_pretrained(PRETRAINED_MODEL, do_lower_case=False)
tokenized_texts_train = [tokenizer.tokenize(sent) for sent in sentences_train]
tokenized_texts_test = [tokenizer.tokenize(sent) for sent in sentences_test]
tokenized_texts_dev = [tokenizer.tokenize(sent) for sent in sentences_dev]

print(sentences_train[0])
print(tokenized_texts_train[0])

In [ ]:
if MAX_LEN == 0:
    str_len = []
    for s in tokenized_texts_train:
        str_len.append(len(s))
    for s in tokenized_texts_test:
        str_len.append(len(s))
    for s in tokenized_texts_dev:
        str_len.append(len(s))
        
    MAX_LEN = max(str_len)
    print('tokenized text의 최대 길이 : ',MAX_LEN)

In [ ]:
input_ids_train = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_train]
input_ids_train = pad_sequences(input_ids_train, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids_test = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_test]
input_ids_test = pad_sequences(input_ids_test, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids_dev = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts_dev]
input_ids_dev = pad_sequences(input_ids_dev, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

input_ids_train[0]

In [ ]:
def set_attention_masks(input_ids):
    attention_masks = []

    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
      
    return attention_masks

attention_masks_train = set_attention_masks(input_ids_train)
attention_masks_test = set_attention_masks(input_ids_test)
attention_masks_dev = set_attention_masks(input_ids_dev)

print(attention_masks_train[0])

In [ ]:
train_inputs = torch.tensor(input_ids_train)
train_labels = torch.tensor(labels_train)
train_masks = torch.tensor(attention_masks_train)
validation_inputs = torch.tensor(input_ids_dev)
validation_labels = torch.tensor(labels_dev)
validation_masks = torch.tensor(attention_masks_dev)

print(train_inputs.size())
print(train_labels.size())
print(train_masks.size())
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

In [ ]:
batch_size = BATCH_SIZE

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = RandomSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
test_inputs = torch.tensor(input_ids_test)
test_labels = torch.tensor(labels_test)
test_masks = torch.tensor(attention_masks_test)

test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

In [ ]:
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

In [ ]:
model = BertForSequenceClassification.from_pretrained(PRETRAINED_MODEL, num_labels=8)
model.cuda()

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

In [ ]:
eepochs = EPOCHS

total_steps = len(train_dataloader) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):

    elapsed_rounded = int(round((elapsed)))

    return datetime.timedelta(seconds=elapsed_rounded)

In [ ]:
seed_val = SEED

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model.zero_grad()

for epoch_i in range(0, epochs):
    

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

  
    t0 = time.time()

    
    total_loss = 0

    
    model.train()
        
    
    for step, batch in enumerate(train_dataloader):
        
        if step % 100 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        
        batch = tuple(t.to(device) for t in batch)
        
        
        b_input_ids, b_input_mask, b_labels = batch

             
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        
        loss = outputs[0]

        
        total_loss += loss.item()

       
        loss.backward()

        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        
        optimizer.step()

       
        scheduler.step()

        
        model.zero_grad()

    
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        

    print("")
    print("Running Validation...")

    
    t0 = time.time()

    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    for batch in validation_dataloader:
        
        batch = tuple(t.to(device) for t in batch)
        
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():     
            
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        logits = outputs[0]

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")

In [ ]:
t0 = time.time()

model.eval()

eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

test_logits = []
test_label_ids = []

for step, batch in enumerate(test_dataloader):
 
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    batch = tuple(t.to(device) for t in batch)

    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():     
     
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
   
    logits = outputs[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    for logit in logits:
        test_logits.append(logit)

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

test_logits = np.array(test_logits)
test_label_ids = np.array(test_label_ids)


print("")
if exam_mode == False: 
    print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))

In [ ]:
import csv

if exam_mode == True: 

    result = np.argmax(test_logits, axis=1).flatten()

    output = []
    output.append(['Id', 'Predicted'])
    for i in range(len(result)):
        output.append([i, inttolabel(result[i])])

    with open('output_en.csv', 'w') as csv_file: 
        csv_w = csv.writer(csv_file, delimiter=",")
        for row in output: 
            csv_w.writerow(row)
        print("결과 저장 완료: ", datetime.datetime.now().strftime("%Y/%m/%d %H:%M:%S"))